# Notebook for deploying on aws and testing model

BLABLABALBALBA Lorem Ipsum BALBALABLAB

---

In [1]:
# import aws sagemaker apis
import boto3
import sagemaker
from sagemaker import get_execution_role

In [2]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()

In [3]:
# specify where to upload in S3
data_dir = './data/dataset/train'
prefix = 'dog-breed-data'

s3_keys = [obj.key for obj in boto3.resource('s3').Bucket(bucket).objects.all()]
# upload to S3 - Due to the size of the dataset it takes some time to finish...
if any(prefix in s for s in s3_keys):
    input_data = f"s3://{bucket}/{prefix}"
else:
    input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-sa-east-1-031843945636/dog-breed-data


## PyTorch Estimator

In [11]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path
# prefix is specified above
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='source', # this should be just "source" for your code
                    role=role,
                    framework_version='1.0',
                    train_instance_count=1,
                    train_instance_type='ml.c5.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_dim': 3,  # num of features
                        'hidden_dim': 256,
                        'output_dim': 119,
                        'epochs': 5, # could change to higher
                        'batch-size': 128
                    })

In [12]:
estimator.fit({'train': input_data})

2020-04-14 15:11:19 Starting - Starting the training job...
2020-04-14 15:11:39 Starting - Launching requested ML instances.........
2020-04-14 15:13:09 Starting - Preparing the instances for training......
2020-04-14 15:13:54 Downloading - Downloading input data....

KeyboardInterrupt: 

In [ ]:
output_path